# Evaluation of a selection of models on 60km -> 2.2km-4x over Birmingham

In [ ]:
%reload_ext autoreload

%autoreload 2

%reload_ext dotenv
%dotenv

import functools
import math
import string

import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xarray as xr

from mlde_analysis.utils import chained_groupby_map
from mlde_analysis.data import prep_eval_data
from mlde_analysis import plot_map, SUBREGIONS
from mlde_analysis.bootstrap import resample_examples
from mlde_analysis.ccs import compute_changes, plot_changes, plot_tp_fd, plot_hist_per_tp, plot_ccs_fc_figure, ccs_fc_da
from mlde_analysis.distribution import PER_GRIDBOX_QUANTILES, normalized_mean_bias, normalized_std_bias, plot_freq_density, plot_mean_biases, plot_std_biases, rms_mean_bias, rms_std_bias, compute_metrics, DIST_THRESHOLDS
from mlde_analysis.fractional_contribution import compute_fractional_contribution, frac_contrib_change, fc_bins
from mlde_analysis.wet_dry import threshold_exceeded_prop_stats, threshold_exceeded_prop_change, plot_threshold_exceedence_errors, THRESHOLDS
from mlde_analysis.display import VAR_RANGES, pretty_table
from mlde_utils import cp_model_rotated_pole, TIME_PERIODS

In [ ]:
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from mlde_analysis.default_params import *

In [ ]:
IPython.display.Markdown(desc)

In [ ]:
EVAL_DS, MODELS = prep_eval_data(sample_configs, dataset_configs, derived_variables_config, eval_vars, split, ensemble_members=ensemble_members, samples_per_run=samples_per_run)
EVAL_DS

In [ ]:
CPM_DAS = { var: EVAL_DS["CPM"][f"target_{var}"] for var in eval_vars }

PRED_DAS = { var: xr.concat([ ds[f"pred_{var}"] for ds in EVAL_DS.values() ], dim="model") for var in eval_vars }

MODELLABEL2SPEC = { model: {"source": source} | spec for source, models in MODELS.items() for model, spec in models.items() } | {"CPM": {"source": "CPM", "color": "black"}}

In [ ]:
CCS_SEASONS = ["DJF", "JJA"]

In [ ]:
CCS_MODELS = {source: {model: spec for model, spec in mconfigs.items() if spec.get("CCS", False)} for source, mconfigs in MODELS.items() }

CCS_PRED_DAS = {var: PRED_DAS[var].sel(model=list([model for source, models in CCS_MODELS.items() for model in models])) for var in eval_vars }

CCS_DS = xr.combine_by_coords([*list(CPM_DAS.values()), *list(CCS_PRED_DAS.values())])

## Figure: per Time Period per season frequency density

* Frequency Density Histogram of rainfall intensities for each time period

In [ ]:
for var in eval_vars:
    fig = plt.figure(layout='constrained', figsize=(3.5*len(CCS_SEASONS), 2*len(TIME_PERIODS)))
    spec = np.array(list([[f"{season} {tp_key}" for season in CCS_SEASONS] for tp_key in TIME_PERIODS.keys() ]))
    
    axd = fig.subplot_mosaic(spec, sharex=True)
    for season, seasonal_ccs_ds in CCS_DS.groupby("time.season"):
        if season not in CCS_SEASONS:
            continue
        
        for tp_key, tp_ds in seasonal_ccs_ds.groupby("time_period"):
            ax = axd[f"{season} {tp_key}"]
            hist_data = [ dict(data=model_ds[f"pred_{var}"], label=model, color=MODELLABEL2SPEC[model]["color"]) for model, model_ds in tp_ds.groupby("model") ]
            hrange=VAR_RANGES[var]
            plot_freq_density(hist_data, ax=ax, target_da=tp_ds[f"target_{var}"], legend=(tp_key=="historic" and season=="DJF"), linewidth=1, title=f"{season} {tp_key}", hrange=hrange)
    
            ax.xaxis.label.set_visible(tp_key == "future")    
            
    plt.show()

### Per time period per season metrics

In [ ]:
for var in eval_vars:
    IPython.display.display_markdown(f"### {var}", raw=True)

    metrics_ds = CCS_DS.groupby("time.season").map(lambda season_ds: season_ds.groupby("time_period").map(lambda tp_ds: compute_metrics(tp_ds[f"pred_{var}"], tp_ds[f"target_{var}"], thresholds=DIST_THRESHOLDS[var])))
        
    pretty_table(metrics_ds, round=4, dim_order=["season", "time_period", "model"], caption="Distribution metrics per season and time period")

## Change in mean

### Mean change maps

In [ ]:
for var in eval_vars:
    IPython.display.display_markdown(f"#### {var}", raw=True)    
    for model, ccs_pred_da in CCS_PRED_DAS[var].groupby("model"):
        changes = compute_changes([ccs_pred_da.squeeze("model")], CPM_DAS[var], CCS_SEASONS, stat_func=xr.DataArray.mean)
        change_fig = plt.figure(figsize=(5.5, 4.5), layout="compressed")
        plot_changes(changes, CCS_SEASONS, change_fig, show_change=[model])
        
    plt.show()

### CCS mean Variablity

## Change in extremes

### High quantile change maps

In [ ]:
for var in eval_vars:
    for q in PER_GRIDBOX_QUANTILES:
        IPython.display.display_markdown(f"#### {var} {q}", raw=True)
        for model, ccs_pred_da in CCS_PRED_DAS[var].groupby("model"):
            changes = compute_changes([ccs_pred_da.squeeze("model")], CPM_DAS[var], CCS_SEASONS, stat_func=functools.partial(xr.DataArray.quantile, q=q))
            change_fig = plt.figure(figsize=(5.5, 4.5), layout="compressed")
            plot_changes(changes, CCS_SEASONS, change_fig, show_change=[model])
            
        plt.show()

### Between samples

i.e. for emulator compute quantiles separately for all sets of samples

In [ ]:
for var in eval_vars:
    IPython.display.display_markdown(f"### {var}", raw=True)
    
    quantiles_ds = xr.merge([
        chained_groupby_map(
            CCS_DS[f"pred_{var}"], 
            ["model", "time.season", "time_period"], 
            lambda gda: gda.quantile(q=PER_GRIDBOX_QUANTILES, dim=["ensemble_member", "time"])
        ).rename(f"pred_{var}_q"),
        chained_groupby_map(
            CCS_DS[f"target_{var}"], 
            ["time.season", "time_period"],
            lambda gda: gda.quantile(q=PER_GRIDBOX_QUANTILES, dim=["ensemble_member", "time"])
        ).rename(f"target_{var}_q")],

    )
    
    quantiles_change_ds = (quantiles_ds.sel(time_period="future") - quantiles_ds.sel(time_period="historic"))

    quantiles_rel_change_ds = 100*(quantiles_change_ds / quantiles_ds.sel(time_period="historic").drop("time_period"))

    mean_q_rel_change_ds = quantiles_rel_change_ds.mean(dim=["grid_longitude", "grid_latitude"])
    
    pretty_table(
        xr.merge([
            xr.concat([
                mean_q_rel_change_ds[f"pred_{var}_q"].mean(dim=["sample_id"]),
                mean_q_rel_change_ds[f"target_{var}_q"].expand_dims(model=["CPM"]),
            ], dim="model").rename("Mean relative change [%]"),
            mean_q_rel_change_ds[f"pred_{var}_q"].min(dim=["sample_id"]).rename("Min mean rel. change over sample_id [%]"),
            mean_q_rel_change_ds[f"pred_{var}_q"].max(dim=["sample_id"]).rename("Max mean rel. change over sample_id [%]"),
            mean_q_rel_change_ds[f"pred_{var}_q"].std(dim=["sample_id"]).rename("Std mean rel. change over sample_id [%]"),

        # xr.merge([quantiles_da.sel(time_period="historic", model="CPM").rename("historic CPM"), quantiles_change_da, quantiles_rel_change_da]),
        ]),
        round=1, 
        caption="Seasonal quantile change",
        dim_order=["season", "quantile", "model"],
    )

### Across samples

i.e. compute quantiles using all samples lumped together for diffusion models

In [ ]:
for var in eval_vars:
    IPython.display.display_markdown(f"### {var}", raw=True)
    
    quantiles_da = xr.concat([
        chained_groupby_map(
            CCS_DS[f"pred_{var}"], 
            ["model", "time.season", "time_period"], 
            lambda gda: gda.quantile(q=PER_GRIDBOX_QUANTILES, dim=["ensemble_member", "time", "sample_id"])
        ),
        chained_groupby_map(
            CCS_DS[f"target_{var}"], 
            ["time.season", "time_period"],
            lambda gda: gda.quantile(q=PER_GRIDBOX_QUANTILES, dim=["ensemble_member", "time"])
        ).expand_dims(model=["CPM"])],
        dim="model",
    ).rename("quantile value")
    
    quantiles_change_da = (quantiles_da.sel(time_period="future") - quantiles_da.sel(time_period="historic")).rename("Change [mm/day]")

    quantiles_rel_change_da = 100*(quantiles_change_da / quantiles_da.sel(time_period="historic").drop("time_period")).rename("Relative Change [%]")
    pretty_table(
        xr.merge([
            quantiles_change_da.mean(dim=["grid_longitude", "grid_latitude"]).rename("mean change [mm/day]"),
            quantiles_rel_change_da.mean(dim=["grid_longitude", "grid_latitude"]).rename("mean relative change [%]"),
            quantiles_rel_change_da.std(dim=["grid_longitude", "grid_latitude"]).rename("std relative change [%]"),
        ]),
        # xr.merge([quantiles_da.sel(time_period="historic", model="CPM").rename("historic CPM"), quantiles_change_da, quantiles_rel_change_da]),
        round=1, 
        caption="Seasonal quantile change bootstrapped spread",
        dim_order=["season", "quantile", "model"],
    )

In [ ]:
for season, season_q_change_da in quantiles_rel_change_da.groupby("season"):
    if season not in CCS_SEASONS:
        continue
    fig, axs = plt.subplots(1, 2, layout="compressed", figsize=(6,3))
    data = season_q_change_da.to_dataframe().reset_index()
    data["Percentile"] = (data["quantile"]*100).astype('category')
    sns.violinplot(data, y="Relative Change [%]", x="Percentile", hue="model", orientation="v", ax=axs[0]).set_title(season)
    sns.boxplot(data, y="Relative Change [%]", x="Percentile", hue="model", ax=axs[1]).set_title(season)
    plt.show()

## Figure: per subdomain per time period per season distribution

In [ ]:
from collections import deque
for var in eval_vars:
    IPython.display.display_markdown(f"### {var}", raw=True)

    for season, seasonal_ccs_ds in CCS_DS.groupby("time.season"):
        if season not in CCS_SEASONS:
            continue

        fig = plt.figure(figsize=(3.5*len(TIME_PERIODS), 2.5*len(SUBREGIONS)), layout="compressed")

        spec = np.array(list([[f"{srname} {tp_key}" for tp_key in TIME_PERIODS.keys()] for srname in SUBREGIONS.keys()]))
        axd = fig.subplot_mosaic(spec, sharex=True)#, sharey=True)
        panel_label_iter = iter(string.ascii_lowercase)
        for srname, srdefn in SUBREGIONS.items():
            for tp_key, tp_ds in seasonal_ccs_ds.groupby("time_period"):
            
                tpsrseason_cpm_da = tp_ds[f"target_{var}"].isel(**SUBREGIONS[srname])
                
                srseason_hist_data = [
                    dict(data=model_ds[f"pred_{var}"].isel(**SUBREGIONS[srname]), label=model, color=MODELLABEL2SPEC[model]["color"]) for model, model_ds in tp_ds.groupby("model")
                ]
    
                ax = axd[f"{srname} {tp_key}"]
                if var == "pr":
                    kwargs = {
                        "yscale": "log",
                    }
                else:
                    kwargs = {
                        "yscale": "linear",
                    }
                plot_freq_density(srseason_hist_data, ax=ax, target_da=tpsrseason_cpm_da, linewidth=1, hrange=VAR_RANGES[var], **kwargs)
                ax.set_title(f"{srname} {season} {tp_key}", size="small")

        deque(axd[axlabel.item()].annotate(f"{next(panel_label_iter)}.", xy=(-0.1, 1.0), xycoords=("axes fraction", "axes fraction"), weight='bold', ha="left", va="bottom") for axlabel in np.nditer(spec))
    
        plt.show()